In [1]:
#Import needed modules
import requests
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3 as lite
import time
from dateutil.parser import parse
import collections
%matplotlib inline

In [2]:
#Pull JSON data from CitiBike website
r = requests.get('http://www.citibikenyc.com/stations/json')

In [4]:
#Pull JSON data into Pandas DataFrame
bikedata=pd.io.json.json_normalize(r.json()['stationBeanList'])

In [5]:
#Build SQLite database with reference table and bike data table
con = lite.connect('citibike.db')
cur = con.cursor()

sql = "INSERT INTO citibike_reference (id, totalDocks, city, altitude, stAddress2, longitude, postalCode, testStation, stAddress1, stationName, landMark, latitude, location) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)"
idList=bikedata['id'].tolist()
idList=['_'+str(x)+' INT' for x in idList]
with con:
    cur.execute('CREATE TABLE citibike_reference (id INT PRIMARY KEY, totalDocks INT, city TEXT, altitude INT, stAddress2 TEXT, longitude NUMERIC, postalCode TEXT, testStation TEXT, stAddress1 TEXT, stationName TEXT, landMark TEXT, latitude NUMERIC, location TEXT)')
    for station in r.json()['stationBeanList']:
        cur.execute(sql,(station['id'],station['totalDocks'],station['city'],station['altitude'],station['stAddress2'],station['longitude'],station['postalCode'],station['testStation'],station['stAddress1'],station['stationName'],station['landMark'],station['latitude'],station['location']))
    
    cur.execute("CREATE TABLE available_bikes (execution_time INT, " + ", ".join(idList)+");")

In [12]:
bikeid=collections.defaultdict(int)

for itr in range(0,60):
    print('Iteration: %d' % itr)
    #Pull execution time
    exec_time = parse(r.json()['executionTime'])

    #Create dictionary for available bikes per station
    for station in r.json()['stationBeanList']:
        bikeid[station['id']] = station['availableBikes']

    with con:
        #Insert execution time into database
        cur.execute('INSERT INTO available_bikes (execution_time) VALUES (?)', (exec_time.strftime("%Y%m%d_%H%M%S"),))

        #Insert available bikes per station into database
        for key,value in bikeid.items():
            cur.execute('UPDATE available_bikes SET _' + str(key) + ' = ' + str(value) + " WHERE execution_time = " + "'"+exec_time.strftime("%Y%m%d_%H%M%S")+"'"+";")
    
    #Pause for 60 seconds till next iteration
    time.sleep(60)

3072
12
2016/07/21_19:00:30
3073
15
2016/07/21_19:00:30
3074
27
2016/07/21_19:00:30
3075
7
2016/07/21_19:00:30
3076
9
2016/07/21_19:00:30
3077
4
2016/07/21_19:00:30
3078
8
2016/07/21_19:00:30
3079
13
2016/07/21_19:00:30
3080
0
2016/07/21_19:00:30
3081
4
2016/07/21_19:00:30
3082
6
2016/07/21_19:00:30
3083
7
2016/07/21_19:00:30
3084
0
2016/07/21_19:00:30
3085
0
2016/07/21_19:00:30
3086
1
2016/07/21_19:00:30
3087
5
2016/07/21_19:00:30
3088
7
2016/07/21_19:00:30
3089
2
2016/07/21_19:00:30
3090
12
2016/07/21_19:00:30
3091
10
2016/07/21_19:00:30
3092
15
2016/07/21_19:00:30
3093
7
2016/07/21_19:00:30
3094
2
2016/07/21_19:00:30
3095
6
2016/07/21_19:00:30
3096
11
2016/07/21_19:00:30
3098
23
2016/07/21_19:00:30
3099
5
2016/07/21_19:00:30
3100
12
2016/07/21_19:00:30
3101
6
2016/07/21_19:00:30
3102
7
2016/07/21_19:00:30
3103
10
2016/07/21_19:00:30
3105
3
2016/07/21_19:00:30
3106
5
2016/07/21_19:00:30
3107
17
2016/07/21_19:00:30
3108
8
2016/07/21_19:00:30
3109
16
2016/07/21_19:00:30
3110
17
2016/07